<a href="https://colab.research.google.com/github/gorzanskik-ai/intro-to-ann/blob/main/05_regression/01_flat_prices_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

np.set_printoptions(precision=12, suppress=True, linewidth=150)
tf.__version__

'2.8.0'

In [7]:
df_raw = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
df = df_raw.copy()
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [10]:
df['ocean_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'], dtype=object)

In [13]:
df.isnull().sum() / len(df) * 100   #braki danych

longitude             0.000000
latitude              0.000000
housing_median_age    0.000000
total_rooms           0.000000
total_bedrooms        1.002907
population            0.000000
households            0.000000
median_income         0.000000
median_house_value    0.000000
ocean_proximity       0.000000
dtype: float64

In [14]:
df.dropna(inplace=True) #usunięcie wierszy z brakami danych

In [15]:
df.isnull().sum() / len(df) * 100   #braki danych

longitude             0.0
latitude              0.0
housing_median_age    0.0
total_rooms           0.0
total_bedrooms        0.0
population            0.0
households            0.0
median_income         0.0
median_house_value    0.0
ocean_proximity       0.0
dtype: float64

In [17]:
df.describe()   #ale uwzględnia tylko kolumny numeryczne

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [19]:
df.describe(include=['object'])     #kolumny kategoryczne

,ocean_proximity
count,20433
unique,5
top,<1H OCEAN
freq,9034


In [21]:
px.histogram(df, x='median_house_value')

In [22]:
df['median_house_value'].value_counts()

500001.0    958
137500.0    119
162500.0    116
112500.0    103
187500.0     92
           ... 
359200.0      1
51200.0       1
39800.0       1
377600.0      1
47000.0       1
Name: median_house_value, Length: 3833, dtype: int64

In [23]:
#usunięcie ceny 500k, ponieważ może zaburzać cały obraz
index_to_drop = df[df['median_house_value'] == 500001.0].index
df = df.drop(index=index_to_drop)
px.histogram(df, x='median_house_value')

In [24]:
#zakodowanie zmiennych kategorycznych za pomoćą get_dummies
df = pd.get_dummies(df, drop_first=True)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,1,0


Zbiór treningowy i testowy

In [37]:
X = df.drop(columns='median_house_value')
y = df['median_house_value']

In [39]:
stats = X.describe().transpose()    #standaryzacja danych

def standarize(x):
    return (x - stats['mean']) / stats['std']

X = standarize(X)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (15580, 12)
X_test shape: (3895, 12)


Budowa modelu

In [45]:
def build_model():
    model = Sequential()
    model.add(Dense(1024, kernel_regularizer='l2', activation='relu', input_shape=(12, )))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mse', 'mae'])
    return model

In [46]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              13312     
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dense_6 (Dense)             (None, 128)               65664     
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 603,905
Trainable params: 603,905
Non-trainable params: 0
_________________________________________________________________


In [47]:
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/150
390/390 [==============================] - 5s 11ms/step - loss: 16555734016.0000 - mse: 16555734016.0000 - mae: 93528.1953 - val_loss: 4373986816.0000 - val_mse: 4373986816.0000 - val_mae: 47742.3828
Epoch 2/150
390/390 [==============================] - 3s 9ms/step - loss: 4055240192.0000 - mse: 4055240192.0000 - mae: 46035.1836 - val_loss: 3529978880.0000 - val_mse: 3529978880.0000 - val_mae: 42446.3633
Epoch 3/150
390/390 [==============================] - 3s 8ms/step - loss: 3672585216.0000 - mse: 3672585216.0000 - mae: 43755.0742 - val_loss: 3384540416.0000 - val_mse: 3384540416.0000 - val_mae: 42049.8906
Epoch 4/150
390/390 [==============================] - 3s 8ms/step - loss: 3544536832.0000 - mse: 3544536832.0000 - mae: 42913.7500 - val_loss: 3379510528.0000 - val_mse: 3379510528.0000 - val_mae: 42426.8359
Epoch 5/150
390/390 [==============================] - 3s 8ms/step - loss: 3473691648.0000 - mse: 3473691648.0000 - mae: 42424.8047 - val_loss: 3318668288.0000 -

In [57]:
def plot_history(history):
    metrics = pd.DataFrame(history.history)
    metrics['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='epochs', yaxis_title='Mean Absolute Error')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['mse'], name='mse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_mse'], name='val_mse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MSE vs. VAL_MSE', xaxis_title='epochs', yaxis_title='Mean Squared Error')
    fig.show()

In [58]:
plot_history(history)

In [60]:
loss, mse, mae = model.evaluate(X_test, y_test)
print(f'loss: {loss}')
print(f'mse: {mse}')
print(f'mae: {mae}')

122/122 [==============================] - 1s 4ms/step - loss: 2323436288.0000 - mse: 2323436288.0000 - mae: 33362.0273
loss: 2323436288.0
mse: 2323436288.0
mae: 33362.02734375


In [64]:
predictions = model.predict(X_test).flatten()

In [66]:
pred_test = pd.DataFrame({'prediction': predictions, 'test': y_test})
pred_test.head()

,prediction,test
14897,93634.546875,96200.0
14605,199273.953125,178700.0
2353,69870.867188,60600.0
11818,188833.453125,191900.0
2881,50208.667969,51000.0


In [70]:
fig = px.scatter(pred_test, 'test', 'prediction')
fig.add_trace(go.Scatter(x=[0, df['median_house_value'].max()], y=[0, df['median_house_value'].max()]))
fig.show()

In [72]:
pred_test['error'] = pred_test['prediction'] - pred_test['test']
pred_test.head()

,prediction,test,error
14897,93634.546875,96200.0,-2565.453125
14605,199273.953125,178700.0,20573.953125
2353,69870.867188,60600.0,9270.867188
11818,188833.453125,191900.0,-3066.546875
2881,50208.667969,51000.0,-791.332031


In [74]:
px.histogram(pred_test, 'error', marginal='rug')